In [80]:
import requests_cache
import numpy as np
import pandas as pd
import os
import re
# from dotenv import load_dotenv
from modules.utils import *

In [15]:
from sqlalchemy import create_engine
engine = create_engine(f"mysql+mariadbconnector://{os.environ['user']}:{os.environ['password']}@{os.environ['host']}:3306/{os.environ['database']}", echo=True)

In [74]:
query = '''
        SELECT *
        FROM stg_addresses
        '''
df = pd.read_sql(query, con=engine)
df

,town,street_name,block_number,full_address
0,ANG MO KIO,ANG MO KIO AVE 1,205,"205, ANG MO KIO AVE 1"
1,ANG MO KIO,ANG MO KIO AVE 1,207,"207, ANG MO KIO AVE 1"
2,ANG MO KIO,ANG MO KIO AVE 1,208,"208, ANG MO KIO AVE 1"
3,ANG MO KIO,ANG MO KIO AVE 1,215,"215, ANG MO KIO AVE 1"
4,ANG MO KIO,ANG MO KIO AVE 1,216,"216, ANG MO KIO AVE 1"
...,...,...,...,...
9518,YISHUN,YISHUN ST 81,876,"876, YISHUN ST 81"
9519,YISHUN,YISHUN ST 81,877,"877, YISHUN ST 81"
9520,YISHUN,YISHUN ST 81,878,"878, YISHUN ST 81"
9521,YISHUN,YISHUN ST 81,879,"879, YISHUN ST 81"


In [19]:
session = requests_cache.CachedSession('logs/address_cache', backend="sqlite")

In [81]:
# Logger here is the debugger logger
logger.info(f"{'-'*50}New run started {'-'*50}")

# Getting latitude, longitude, postal code
def get_lat_long(address_df : pd.DataFrame, verbose: int=1):
    '''
    The actual API call to be called row-wise to get latitude, longitude, and postal code
    ## Parameters
    address_df : pd.DataFrame
        DataFrame that contains a combination of ['block_number'] and ['street_name'] as ['full_address']
    '''

    # API call
    try:
        address = address_df['full_address']
        call = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={address}&returnGeom=Y&getAddrDetails=Y"
        # Caching is enabled in the session
        response = session.get(call)
        response.raise_for_status()
        data = response.json()
        if verbose ==1:
            logger.info(f'Response: {response.status_code} \tGet request call: {response.url}')

        # For those with ST abbreviations
        if len(data['results'])<1:
            address = re.sub(' ST', ' STREET', address_df['full_address'])
            call = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={address}&returnGeom=Y&getAddrDetails=Y"
            # Caching is enabled in the session
            response = session.get(call)
            response.raise_for_status()
            data = response.json()
            if verbose ==1:
                logger.info(f'Response: {response.status_code} \tGet request call: {response.url}')

        return [float(data['results'][0]['LATITUDE']), float(data['results'][0]['LONGITUDE']), data['results'][0]['POSTAL']]

    except Exception as err:
        logger.error(f'Error occurred - get_lat_long() API call: {err} on the following call:', exc_info=True)
        return [np.NaN, np.NaN, np.NaN ]

# This calls the API call function row wise
df[['latitude', 'longitude', 'postcode']] = df.apply(get_lat_long, axis=1, result_type='expand')

In [84]:
df.to_sql("geolocations", engine, if_exists="replace") 

9523

In [85]:
query = '''
        SELECT *
        FROM geolocations
        LIMIT 200
        '''
df = pd.read_sql(query, con=engine)
df

,index,town,street_name,block_number,full_address,latitude,longitude,postcode
0,0,ANG MO KIO,ANG MO KIO AVE 1,205,"205, ANG MO KIO AVE 1",1.366941,103.843582,560205
1,1,ANG MO KIO,ANG MO KIO AVE 1,207,"207, ANG MO KIO AVE 1",1.365821,103.842848,560207
2,2,ANG MO KIO,ANG MO KIO AVE 1,208,"208, ANG MO KIO AVE 1",1.365445,103.842715,560208
3,3,ANG MO KIO,ANG MO KIO AVE 1,215,"215, ANG MO KIO AVE 1",1.366558,103.841624,560215
4,4,ANG MO KIO,ANG MO KIO AVE 1,216,"216, ANG MO KIO AVE 1",1.366197,103.841505,560216
...,...,...,...,...,...,...,...,...
195,195,ANG MO KIO,ANG MO KIO AVE 3,570,"570, ANG MO KIO AVE 3",1.369744,103.856023,560570
196,196,ANG MO KIO,ANG MO KIO AVE 3,571,"571, ANG MO KIO AVE 3",1.370055,103.854881,560571
197,197,ANG MO KIO,ANG MO KIO AVE 3,572,"572, ANG MO KIO AVE 3",1.370258,103.855969,560572
198,198,ANG MO KIO,ANG MO KIO AVE 3,573,"573, ANG MO KIO AVE 3",1.370561,103.854823,560573
